### 启动spark运行环境

In [1]:
import os
##下面这个语句，我有印象，好像是为了满足某些shell的什么东西吧，但是删除了依然好使
os.environ['PYSPARK_PYTHON']='/usr/bin/python2' 
spark_home = os.getenv("SPARK_HOME")

#将pyspark的位置增加到python的库搜索路径
import sys
sys.path.insert(0,os.path.join(spark_home,"python"))

#获取sparkSession
from pyspark.sql import SparkSession
def get_spark_session(app_name = None, master = "spark://master:7077"):
    spark = SparkSession.builder\
        .master(master) \
        .appName(app_name) \
        .config("ui.showConsoleProgress","true") \
        .config("spark.executor.cores","3") \
        .config("spark.executor.memory","10G") \
        .getOrCreate()
    return spark


import time
now_date_str = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))


Your_app_name = ""
spark = get_spark_session(Your_app_name + now_date_str)

### 加载数据

In [43]:
iris_data = spark.read.options(inferschema='true').csv("/VChao/mllib_test/iris.data")
iris_data.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)



### 预处理数据

#### 为类别进行编号

In [14]:
from pyspark.ml.feature import StringIndexer

In [44]:
index_model = StringIndexer(inputCol="_c4", outputCol="class").fit(iris_data)
iris_data = index_model.transform(iris_data)

In [45]:
iris_data = iris_data.drop("_c4").cache()

#### 处理为机器学习算法可使用的数据格式

In [21]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

data = iris_data.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[:-1])))

#### 按照比例切分数据 

In [37]:
train_ratio = 0.8
splits =[train_ratio, 1 - train_ratio]
train_data,test_data = data.randomSplit(splits)

### 选择算法并训练

In [38]:
from pyspark.mllib.tree import RandomForest
rf_model =RandomForest.trainClassifier(train_data,numClasses=3,numTrees=10, \
                                       categoricalFeaturesInfo={},featureSubsetStrategy='auto',impurity="gini")

### 评估算法性能

In [46]:
pre = predictions.collect()

predictions = rf_model.predict(test_data.map(lambda x:x.features))


np_pre = np.array(pre)
labels = test_data.map(lambda x:x.label).collect()
np_labels = np.array(labels)


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print confusion_matrix(np_labels, np_pre)
print classification_report(np_labels, np_pre,target_names=index_model.labels)

[[4 0 0]
 [0 8 1]
 [0 0 3]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         4
Iris-versicolor       1.00      0.89      0.94         9
 Iris-virginica       0.75      1.00      0.86         3

      micro avg       0.94      0.94      0.94        16
      macro avg       0.92      0.96      0.93        16
   weighted avg       0.95      0.94      0.94        16

